In [111]:
import re
import os
import zipfile
import string

# Regular expressions to extract data from the corpus
doc_regex = re.compile("<DOC>.*?</DOC>", re.DOTALL)
docno_regex = re.compile("<DOCNO>.*?</DOCNO>")
text_regex = re.compile("<TEXT>.*?</TEXT>", re.DOTALL)

In [112]:
with open("stopwords.txt", 'r') as sw:
    stopwords = sw.read().split('\n')

In [113]:
with zipfile.ZipFile("ap89_collection_small.zip", 'r') as zip_ref:
    zip_ref.extractall()
   
# Retrieve the names of all files to be indexed in folder ./ap89_collection_small of the current directory
for dir_path, dir_names, file_names in os.walk("ap89_collection_small"):
    allfiles = [os.path.join(dir_path, filename).replace("\\", "/") for filename in file_names if (filename != "readme" and filename != ".DS_Store")]

In [122]:
terms=[]
docs=[]
index=[]
#allfiles = allfiles[0:2]
for file in allfiles:
    with open(file, 'r', encoding='ISO-8859-1') as f:
        filedata = f.read()
        result = re.findall(doc_regex, filedata)  # Match the <DOC> tags and fetch documents
        
        for document in result[0:]:
            # Retrieve contents of DOCNO tag
            docno = re.findall(docno_regex, document)[0].replace("<DOCNO>", "").replace("</DOCNO>", "").strip()
            # Retrieve contents of TEXT tag
            text = "".join(re.findall(text_regex, document))\
                      .replace("<TEXT>", "").replace("</TEXT>", "")\
                      .replace("\n", " ")

            # step 1 - lower-case words, remove punctuation, remove stop-words, etc., create tokens
            tokens = text.translate(str.maketrans('', '', string.punctuation)).lower().split(' ')
            for word in tokens:
                if word in stopwords:
                    tokens.remove(word)
            
            # step 2 - build index
            docs.append((docno,len(tokens)))
            for i in range(len(tokens)):
                curr = tokens[i]
                if curr not in terms:
                    terms.append(curr)
                    newEntry = {"termid": terms.index(curr),
                                "instances": [(docs.index((docno,len(tokens))), i)]}
                    index.append(newEntry)
                else:
                    index[terms.index(curr)]["instances"].append((docs.index((docno,len(tokens))),i))
                

#clear / write to docids.txt
#each line contains tuple(DOCID FILENAME TOKENCOUNT)
docids = open("docids.txt","r+")
docids.truncate(0)
for i in range(len(docs)):
    newline = str(i)+"\t"+docs[i][0]+"\t"+str(docs[i][1])+"\n"
    docids.write(newline)
docids.close()

#clear / write to termids.txt
#each line contains tuple(TERMID TOKEN)
termids = open("termids.txt","r+")
termids.truncate(0)
for i in range(len(terms)):
    newline = str(i)+"\t"+terms[i]+"\n"
    termids.write(newline)
termids.close()

#clear / write to term_index.txt
#each line contains format TERMID DOCID:POSITION DOCID:POSITION
term_index = open("term_index.txt","r+")
term_index.truncate(0)
for i in range(len(index)):
    newline = str(index[i]["termid"])
    for instance in index[i]["instances"]:
        newline += "\t"+str(instance[0])+":"+str(instance[1])
    term_index.write(newline+"\n")
term_index.close()

#clear / write to term_info.txt
# TERMID BYTEOFFSET NUMOCCUR DOCSWITH
term_info = open("term_info.txt","r+")
term_info.truncate(0)
term_index = open("term_index.txt","r")
while term_index:
    tell = term_index.tell()
    linestr = term_index.readline()
    if linestr == "":
        break
    line = linestr.split("\t")
    newline = str(line[0])+"\t"+str(tell)+"\t"
    count=0
    newdocs=0
    lastdoc=-1
    for i in line[1:]:
        count+=1
        docid = i.split(":")[0]
        if docid != lastdoc:
            newdocs+=1
            lastdoc=docid
    newline += str(count)+"\t"+str(newdocs)
    term_info.write(newline+"\n")
term_info.close()
term_index.close()
